In [1]:
import ikpy.chain
import math

In [ ]:
# based on https://www.youtube.com/watch?v=XDSzbJAwJKA
# urdf from: https://github.com/cwru-robotics/cwru-ros-pkg/blob/master/catkin/src/cwru_376_student/abby_arm_simu/abb_common/urdf/irb120.urdf

def ik_string(target_position):
    target_orientation = [-1.0, 0.0, 0.0]
    my_chain = ikpy.chain.Chain.from_urdf_file("irb120.urdf", active_links_mask=[False, True, True, True, True, True, True, False]) #Base link not considered active, and the urdf seems to contain a 7th non active link?
    ik = my_chain.inverse_kinematics(target_position, target_orientation, orientation_mode="Y")
    res = list(map(lambda r:math.degrees(r),ik.tolist()))
    string = f"{{'action': 'MoveJ', 'value': {{'joint1': {res[1]}, 'joint2': {res[2]}, 'joint3': {res[3]}, 'joint4': {res[4]}, 'joint5': {res[5]}, 'joint6': {res[6]}}}, 'speed': 1.0}}"
    return string

In [42]:
class object:
    def __init__(self, x, y, z, name):
        self.x = x*1.00
        self.y = y*1.00
        self.z = z*1.00
        self._name = name #NOTE Must match the name assigned to the object in the simulation environment

    def name(self):
        return self._name


class action_module(object):
    '''
    '''


    def initial_module():
        string = r"{'action': 'MoveJ', 'value': {'joint1': 0.0, 'joint2': 0.0, 'joint3': 0.0, 'joint4': 0.0, 'joint5': 0.0, 'joint6': 0.0}, 'speed': 1.0}" #Don't need a test
        return string

    def initialize_ee(ee):
        # ee location based on this function
        ee.x = 0.54545
        ee.y = 0.0007955
        ee.z = 0.62942
        string = r"{'action': 'MoveJ', 'value': {'joint1': -58.10, 'joint2': 25.75, 'joint3': 27.17, 'joint4': -63.59, 'joint5': -71.41, 'joint6': 77.67}, 'speed': 1.0}"
        return string

    def pre_grasp_module(obj,ee):
        # Pre-grasp (0.1 (z-axis) above the object)
        string = f"{{'action': 'MoveL', 'value': {{'movex': {obj.x-ee.x}, 'movey': {obj.y-ee.y}, 'movez': {obj.z-ee.z+0.1}}}, 'speed': 1.0}}"
        ee.x = obj.x-ee.x
        ee.y = obj.y-ee.y
        ee.z = obj.z-ee.z+0.1
        return string

    def move_down_module(ee):
        # Grasp (this sets the values to be inside the object?)
        string = r"{'action': 'MoveL', 'value': {'movex': 0.0, 'movey': 0.0, 'movez': -0.1}, 'speed': 1.0}"
        ee.z = ee.z - 0.1
        return string
    
    def attach_module(obj):
        # Attach #NOTE if this fails in the simulation, perhaps the name of the endeffector is wrong as this
        # assumes that all end_effectors are called EE_egp64
        string = f"{{'action': 'Attach', 'value': {{'object': '{obj.name()}', 'endeffector': 'EE_egp64'}}}}"
        return string
    
    def move_up_module(ee):
        # We need to move up after grasping to avoid collision
        string = r"{'action': 'MoveL', 'value': {'movex': 0.00, 'movey': 0.0, 'movez': 0.1}, 'speed': 1.0}"
        ee.z = ee.z + 0.1
        return string
    
    def detach_module(obj):
        string = f"{{'action': 'Detach', 'value': {{'object': '{obj.name()}'}}}}"
        return string
    
    def close_module():
        return r"{'action': 'GripperClose'}" #Don't need a test
    
    def open_module():
        return r"{'action': 'GripperOpen'}" #Don't need a test
    
    def above_new_position(ee,x,y,z):
        string = f"{{'action': 'MoveL', 'value': {{'movex': {x-ee.x}, 'movey': {y-ee.y}, 'movez': {z-ee.z+0.1}}}, 'speed': 1.0}}"
        ee.x = x-ee.x
        ee.y = y-ee.y
        ee.z = z-ee.z+0.1
        return string
    

In [43]:
# Create an instance of the Object class with coordinates (1, 1, 1)
ee_x = 0.54545
ee_y = 0.0007955
ee_z = 0.62942
ee = object(ee_x,ee_y,ee_z,"ee")
box = object(1,2,3,"box")
# Testing
print(ee.x,ee.y,ee.z)
md = action_module
string = md.pre_grasp_module(box,ee)
print(string)
print(ee.x,ee.y,ee.z)


0.54545 0.0007955 0.62942
{'action': 'MoveL', 'value': {'movex': 0.45455, 'movey': 1.9992045, 'movez': 2.47058}, 'speed': 1.0}
0.45455 1.9992045 2.47058


In [44]:
# Testing if they work correctly
#assert action.initialize_ee(action,item) == ?, "failure"
#action = action_module
#item = object(1,2,3,"box")
#ee = object(ee_x,ee_y,ee_z,"ee")
#assert action.pre_grasp_module(item,ee) == r"{'action': 'MoveXYZ', 'value': {'positionx': 1.0, 'positiony': 2.0, 'positionz': 3.1}, 'speed': 1.0}", "failure"
#assert action.attach_module(item) == r"{'action': 'Attach', 'value': {'object': 'item', 'endeffector': 'EE_egp64'}}"
#assert action.detach_module(item) == r"{'action': 'Detach', 'value': {'object': 'item'}}"
#assert action.above_new_position(x,y,z) == r"{'action': 'MoveXYZ', 'value': {'positionx': 4.0, 'positiony': 5.0, 'positionz': 6.1}, 'speed': 1.0}", "failure"


In [45]:
''' NOT UP-TO-DATE
# Example object:
box = object(1, 1, 0, "box")
# Target position
x1,y1,z1 = 1, 0, 0

# To generate the file:
model = action_module
action_list = []
#init
action_list.append(model.initial_module())
action_list.append(model.initialize_ee())
#pregrasp
action_list.append(model.pre_grasp_module(box))
#grasp
action_list.append(model.move_down_module())
#attach
action_list.append(model.attach_module(box))
#moveup
action_list.append(model.move_up_module())
#newlocation
action_list.append(model.above_new_position(x1,y1,z1))
action_list.append(model.move_down_module())
#detach
action_list.append(model.detach_module(box))
#init
action_list.append(model.move_up_module())
action_list.append(model.initial_module())

action_list
# Name of the text file
filename = 'list_elements.txt'

# Open the file in write mode
with open(filename, 'w') as file:
    # Iterate over the list and write each element to a separate line in the file
    for line in action_list:
        file.write(line + '\n')


'''

' NOT UP-TO-DATE\n# Example object:\nbox = object(1, 1, 0, "box")\n# Target position\nx1,y1,z1 = 1, 0, 0\n\n# To generate the file:\nmodel = action_module\naction_list = []\n#init\naction_list.append(model.initial_module())\naction_list.append(model.initialize_ee())\n#pregrasp\naction_list.append(model.pre_grasp_module(box))\n#grasp\naction_list.append(model.move_down_module())\n#attach\naction_list.append(model.attach_module(box))\n#moveup\naction_list.append(model.move_up_module())\n#newlocation\naction_list.append(model.above_new_position(x1,y1,z1))\naction_list.append(model.move_down_module())\n#detach\naction_list.append(model.detach_module(box))\n#init\naction_list.append(model.move_up_module())\naction_list.append(model.initial_module())\n\naction_list\n# Name of the text file\nfilename = \'list_elements.txt\'\n\n# Open the file in write mode\nwith open(filename, \'w\') as file:\n    # Iterate over the list and write each element to a separate line in the file\n    for line in a

In [46]:
# Based on the above code and instructions, let's generate instructions
# when objects are added to the simulation, they should also be added this dictionary with their position and name,
# otherwise this code wont work. Alternatively, we could add all the objects here manually.
# NOTE: to add
#   - The below directory should be updated once the object moves
#   - A logic for adding and updating the below directory should be explored and added

obj_dic = {}

def generate_instruction_set(instructions, save_file):
    target_object = obj_dic[instructions['what']] # Here we access the object
    task = instructions['how']
    location = instructions['location']
    action_list = []
    model = action_module
    match task:
        case 'move':
            # Currently we only go here
            #init
            action_list.append(model.initial_module())
            action_list.append(model.initialize_ee(obj_dic['ee']))
            #pregrasp
            action_list.append(model.pre_grasp_module(target_object,obj_dic['ee']))
            #grasp
            action_list.append(model.move_down_module(obj_dic['ee']))
            #attach
            action_list.append(model.attach_module(target_object))
            #moveup
            action_list.append(model.move_up_module(obj_dic['ee']))
            #newlocation
            if location['precise'] == 'True':
                # Precise location logic here:
                ""
            else: # Not precise:
                if location['relation'] == 'None':
                    result = move_with_respect_to(target_object, location['direction'], distance_to_float(location['distance']))
                    action_list.append(result[0])
                    action_list.append(result[1])
                else: #We have to move in relation to the other object (location['relation'])
                    result = move_with_respect_to(obj_dic[location['relation']], location['direction'], distance_to_float(location['distance']))
                    action_list.append(result[0])
                    action_list.append(result[1])
            #detach
            action_list.append(model.detach_module(target_object))
            #init
            action_list.append(model.move_up_module(obj_dic['ee']))
            action_list.append(model.initial_module())
        case 'some other action': 
            ""# This is a place-holder for future tasks, but a similar implementation would made here as above

    # Open the file in write mode
    with open(save_file, 'w') as file:
        # Iterate over the list and write each element to a separate line in the file
        for line in action_list:
            file.write(line + '\n')


def distance_to_float(distance):
    ''' 
    distance: String (for example 10 cm)
    return: float (0.1)
    '''
    value, unit = distance.split()
    value = float(value)

    if unit == 'cm':
        value /= 100.0

    return value


def move_with_respect_to(relation, direction, distance):
    '''
        parameter relation: type object (either some other object or itself)
        parameter direction: type string
        parameter action_module: type action_module

        return: String

        This function is basically for generating the target location of a object being moved
        and returning the string that corresponds that action
    '''

    list_of_strings = []
    am = action_module
   
    match direction:
        case 'forward':
            list_of_strings.append(am.above_new_position(obj_dic['ee'],relation.x,relation.y+distance,relation.z))
            list_of_strings.append(am.move_down_module(obj_dic['ee']))
        case 'backwards':
            list_of_strings.append(am.above_new_position(obj_dic['ee'],relation.x,relation.y-distance,relation.z))
            list_of_strings.append(am.move_down_module(obj_dic['ee']))
        case 'right':
            list_of_strings.append(am.above_new_position(obj_dic['ee'],relation.x+distance,relation.y,relation.z))
            list_of_strings.append(am.move_down_module(obj_dic['ee']))
        case 'left':
            list_of_strings.append(am.above_new_position(obj_dic['ee'],relation.x-distance,relation.y,relation.z))
            list_of_strings.append(am.move_down_module(obj_dic['ee']))
        case 'behind': #same as backwards but a different object will be passed to the function
            list_of_strings.append(am.above_new_position(obj_dic['ee'],relation.x,relation.y-distance,relation.z))
            list_of_strings.append(am.move_down_module(obj_dic['ee']))
        case 'in front': # same as forward but a different object will be passed to the function
            list_of_strings.append(am.above_new_position(obj_dic['ee'],relation.x,relation.y+distance,relation.z))
            list_of_strings.append(am.move_down_module(obj_dic['ee']))
        case '':
            ""
    
    return list_of_strings
        

In [47]:
distance_to_float("1 m")

1.0

In [49]:
# Example instruction
example_instructions = {'what': 'cup',
    'how': 'move',
    'location': {'precise': 'False',
     'relation': 'box',
     'direction': 'forward',
     'distance': '10 cm'}
}
# Create the objects:
cup = object(1,1,0,'cup')
box = object(1,0,0,'box')
# Create the end-effector
ee_x = 0.54545
ee_y = 0.0007955
ee_z = 0.62942
ee = object(ee_x,ee_y,ee_z,"ee")
# Add to dictionary:
obj_dic['cup'] = cup
obj_dic['box'] = box
obj_dic['ee'] = ee
# Now let's generate instruction based on the above instructions:
generate_instruction_set(example_instructions, 'test_file1.txt')